In [1]:
import os
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float64)

import botorch
from botorch.acquisition import ExpectedImprovement, NoisyExpectedImprovement,\
    PosteriorMean, UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import qExpectedImprovement, qProbabilityOfImprovement, qUpperConfidenceBound
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning

import time
import warnings
from botorch.acquisition.objective import ConstrainedMCObjective
from botorch.test_functions import Hartmann, Ackley, EggHolder, Cosine8, DixonPrice, Griewank, Levy, Powell

from botorch.models import SingleTaskGP, FixedNoiseGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.kernels import MaternKernel

from scipy.stats.qmc import Sobol
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, average_precision_score

test_datasets = {
    'iris':{
        'label':'Class',
        'problem':'classification',
        'features':['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'],
        'filepath':'../data/iris_processed.csv'
    },
    'car':{
        'label':'class',
        'problem':'classification',
        'features':['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'],
        'filepath':'../data/car_processed.csv'
    },
    'drybeans':{
        'label':'Class',
        'problem':'classification',
        'features':['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
           'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
           'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
           'ShapeFactor3', 'ShapeFactor4'],
        'filepath':'../data/drybeans_processed.csv'
    },
    'adult':{
        'label':'income-class',
        'problem':'classification',
        'features':['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
                'hours-per-week', 'native-country'],
        'filepath':'../data/adult_processed.csv'
    },
    'diabetes':{
        'label':'readmitted',
        'problem':'classification',
        'features':['patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id',
            'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
            'payer_code', 'medical_specialty', 'num_lab_procedures',
            'num_procedures', 'num_medications', 'number_outpatient',
            'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
            'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
            'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
            'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
            'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
            'tolazamide', 'examide', 'citoglipton', 'insulin',
            'glyburide-metformin', 'glipizide-metformin',
            'glimepiride-pioglitazone', 'metformin-rosiglitazone',
            'metformin-pioglitazone', 'change', 'diabetesMed'],
        'filepath':'../data/diabetes_processed.csv'
    }
}

hyperparam_bounds = [
    {'name':'n_estimators',
     'bounds':torch.tensor([[1], [100]]),
     'round':True},
    {'name':'max_depth',
     'bounds':torch.tensor([[1], [10]]),
     'round':True},
    {'name':'max_leaves',
     'bounds':torch.tensor([[64], [2048]]),
     'round':True},
    {'name':'grow_policy',
     'bounds':torch.tensor([[0], [1]]),
     'round':True},
    {'name':'learning_rate',
     'bounds':torch.tensor([[0.01], [2]]),
     'round':False},
    {'name':'gamma',
     'bounds':torch.tensor([[0], [20]]),
     'round':False},
    {'name':'subsample',
     'bounds':torch.tensor([[0], [1]]),
     'round':False},
    {'name':'colsample_bytree',
     'bounds':torch.tensor([[0], [1]]),
     'round':False},
    {'name':'colsample_bylevel',
     'bounds':torch.tensor([[0], [1]]),
     'round':False},
    {'name':'colsample_bynode',
     'bounds':torch.tensor([[0], [1]]),
     'round':False}
]
for i in range(len(hyperparam_bounds)):
    hyperparam_bounds[i]['bounds'] = hyperparam_bounds[i]['bounds'].double()
    
def map_to_hyperparams(t):
    """
    Map the candidate tensor to the hyperparam space
    """
    t = t.T
    hyperparams = {}
    for i in range(len(t)):
        info = hyperparam_bounds[i]
        hyperparams[info['name']] = t[i] * (info['bounds'][1] - info['bounds'][0]) + info['bounds'][0]
        if info['round']:
            hyperparams[info['name']] = hyperparams[info['name']].round()
        hyperparams[info['name']] = float(hyperparams[info['name']])
        if info['round']:
            hyperparams[info['name']] = int(hyperparams[info['name']])
        if info['name'] == 'grow_policy':
            if hyperparams[info['name']] == 0:
                hyperparams[info['name']] = 'depthwise'
            if hyperparams[info['name']] == 1:
                hyperparams[info['name']] = 'lossguide'
    return hyperparams
    
models = {
    'matern2_5':{
        'gp':SingleTaskGP,
        'likelihood':ExactMarginalLogLikelihood,
        'covar':MaternKernel(2.5) # Matern by default
    },
    'matern1_5':{
        'gp':SingleTaskGP,
        'likelihood':ExactMarginalLogLikelihood,
        'covar':MaternKernel(nu=1.5) # Matern by default
    },
    'matern0_5':{
        'gp':SingleTaskGP,
        'likelihood':ExactMarginalLogLikelihood,
        'covar':MaternKernel(nu=0.5) # Matern by default
    }
}

def initialize_first_batch(bounds, m=3):
    """
    bounds: the range in which we want to generate points.
    m: the power of 2 representing the number of Sobol points to generate.
    """
    d = bounds.shape[1]
    sobol_points = torch.tensor(Sobol(d=d, scramble=False).random_base2(m=m))
    # map each dimension to the bounds
    j = 0
    for i in bounds.T:
        sobol_points.T[j] = (i[1] - i[0]) * sobol_points.T[j] + i[0]
        j += 1
    return sobol_points

def prep_model(models, model_key, X_train, y_train, state_dict=None):
    """
    Initialize model according to specifications from the model dict and training data
    Hotload the params from state_dict to speed up likelihood fitting
    """
    model = models[model_key]['gp'](X_train, y_train, covar_module=models[model_key]['covar'])
    if state_dict is not None:
        model.load_state_dict(state_dict)
        
    mll = models[model_key]['likelihood'](model.likelihood, model)
    return mll, model

def double_train_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    return X_train, X_val, X_test, y_train, y_val, y_test

def train_xgb(X_train, y_train, hyperparams):
    model = xgb.XGBClassifier(**hyperparams)
    model.fit(X_train, y_train)
    return model
    
def evaluate_xgb(X_val, y_val, model):
    preds = model.predict_proba(X_val)
    preds = preds[np.arange(len(preds)), y_val.astype(int)]
    return torch.tensor(preds.mean()).unsqueeze(-1)

def training_loop(X_train, y_train, X_val, y_val, dataset_key, 
                  models, model_key, acq_function,
                  loops, initialization_count_m=2, q=4):
    """
    Trains BO model on function defined by specified key for specified loop count
    """
    # initialize data
    bo_train = initialize_first_batch(torch.tensor([[0] * 10, [1] * 10]), m=initialization_count_m)
    bo_val = torch.tensor([])
    for i in range(len(bo_train)):
        hyperparams = bo_train[i]
        hyperparams = map_to_hyperparams(hyperparams)
        xgb_model = train_xgb(X_train, y_train, hyperparams)
        f1 = evaluate_xgb(X_val, y_val, xgb_model)
        bo_val = torch.cat([bo_val, f1])
    
    mll, model = prep_model(models=models, model_key=model_key, X_train=bo_train, y_train=bo_val.unsqueeze(-1))
    
    # fit generate update loop
    for i in range(loops):
        # reload model
        mll, model = prep_model(models=models, 
            model_key=model_key, 
            X_train=bo_train, 
            y_train=bo_val.unsqueeze(-1),
            state_dict=model.state_dict())
        # fit model and acquisition function
        fit_gpytorch_mll(mll)
        best_value = bo_val.max()
        
        sampler = SobolQMCNormalSampler(1024)
        if acq_function == qExpectedImprovement:
            acq = acq_function(model=model, best_f=best_value, sampler=sampler, maximize=True)
        if acq_function == qProbabilityOfImprovement:
            acq = acq_function(model=model, best_f=best_value, sampler=sampler)
        if acq_function == qUpperConfidenceBound:
            acq = acq_function(model=model, beta=0.2, sampler=sampler)
            
        # generate new points
        candidates, _ = optimize_acqf(
            acq_function=acq,
            bounds=torch.tensor([[0] * 10, [1] * 10]).double(),
            q=q,
            num_restarts=10,
            raw_samples=10,  # used for initialization heuristic
        )
        # map candidates to their respective space
        for i in range(len(candidates)):
            hyperparams = candidates[i]
            hyperparams = map_to_hyperparams(hyperparams)
            print('Training model') 
            print(hyperparams)
            xgb_model = train_xgb(X_train, y_train, hyperparams)
            f1 = evaluate_xgb(X_val, y_val, xgb_model)
            bo_val = torch.cat([bo_val, f1])
            bo_train = torch.cat([bo_train, candidates[i].unsqueeze(0)])
            print('')
        best_value = bo_val.max()
        
    return mll, model, best_value

In [ ]:
performance = []

for dataset_key in test_datasets.keys():
    df = pd.read_csv(test_datasets[dataset_key]['filepath'])
    X = df[test_datasets[dataset_key]['features']]
    y = df[test_datasets[dataset_key]['label']]
    
    X_train, X_val, X_test, y_train, y_val, y_test = double_train_test_split(X, y)

    for model_key in models.keys():
        for q in [1,4,16]:  
            for acq_function in [qExpectedImprovement, qProbabilityOfImprovement, qUpperConfidenceBound]:
                if acq_function == qExpectedImprovement:
                    acq_function_name = 'Expected Improvement'
                if acq_function == qProbabilityOfImprovement:
                    acq_function_name = 'Probability of Improvement'
                if acq_function == qUpperConfidenceBound:
                    acq_function_name = 'Upper Confidence Bound'

                print('attempting - dataset {} model {} q {} acq {}'\
                    .format(dataset_key, model_key, q, acq_function))
                loops = 32 // q
                mll, model, best_value = training_loop(X_train, y_train, X_val, y_val, dataset_key=dataset_key,
                      models=models, model_key=model_key, acq_function=acq_function,
                      loops=loops, q=q)

                performance.append({
                    'Model':model_key,
                    'Dataset':dataset_key,
                    'Q':q,
                    'Acquisition Function':acq_function_name,
                    'Performance':float(best_value)
                })
                print(best_value)
                print('')

attempting - dataset iris model matern2_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
Training model
{'n_estimators': 42, 'max_depth': 10, 'max_leaves': 1602, 'grow_policy': 'lossguide', 'learning_rate': 0.8166569521278143, 'gamma': 6.751042436808348, 'subsample': 0.34844369627535343, 'colsample_bytree': 0.36505372542887926, 'colsample_bylevel': 0.9828666057437658, 'colsample_bynode': 0.02577290590852499}

Training model
{'n_estimators': 2, 'max_depth': 2, 'max_leaves': 1455, 'grow_policy': 'depthwise', 'learning_rate': 1.8587457980401814, 'gamma': 13.01481232047081, 'subsample': 0.803204563446343, 'colsample_bytree': 0.09634192381054163, 'colsample_bylevel': 0.030782559886574745, 'colsample_bynode': 0.94606311339885}

Training model
{'n_estimators': 86, 'max_depth': 9, 'max_leaves': 1754, 'grow_policy': 'lossguide', 'learning_rate': 0.18750248133204878, 'gamma': 0.46313880011439323, 'subsample': 0.2767493147403002, 'colsample_bytree': 0.07076558470726013, 'c


Training model
{'n_estimators': 60, 'max_depth': 4, 'max_leaves': 760, 'grow_policy': 'depthwise', 'learning_rate': 1.7746106576826424, 'gamma': 2.4750902876257896, 'subsample': 0.3017394943162799, 'colsample_bytree': 0.9537141630426049, 'colsample_bylevel': 0.3163217790424824, 'colsample_bynode': 0.7123163798823953}

Training model
{'n_estimators': 95, 'max_depth': 9, 'max_leaves': 775, 'grow_policy': 'lossguide', 'learning_rate': 0.1627445133123547, 'gamma': 2.316997852176428, 'subsample': 0.5526400292292237, 'colsample_bytree': 0.9089867547154427, 'colsample_bylevel': 0.33779815677553415, 'colsample_bynode': 0.6058513559401035}

Training model
{'n_estimators': 63, 'max_depth': 5, 'max_leaves': 1461, 'grow_policy': 'depthwise', 'learning_rate': 1.8797462764382362, 'gamma': 1.9337339513003826, 'subsample': 0.03879047930240631, 'colsample_bytree': 0.29515683371573687, 'colsample_bylevel': 0.9379037469625473, 'colsample_bynode': 0.16185113228857517}

Training model
{'n_estimators': 58,


Training model
{'n_estimators': 53, 'max_depth': 2, 'max_leaves': 1958, 'grow_policy': 'depthwise', 'learning_rate': 0.7473065948300064, 'gamma': 1.164022870361805, 'subsample': 0.7035737568512559, 'colsample_bytree': 0.14927495550364256, 'colsample_bylevel': 0.9161569373682141, 'colsample_bynode': 0.49204817973077297}

Training model
{'n_estimators': 62, 'max_depth': 1, 'max_leaves': 940, 'grow_policy': 'depthwise', 'learning_rate': 0.20602693978697062, 'gamma': 13.583436738699675, 'subsample': 0.9893764518201351, 'colsample_bytree': 0.20222554542124271, 'colsample_bylevel': 0.9908813843503594, 'colsample_bynode': 0.9080171342939138}

Training model
{'n_estimators': 69, 'max_depth': 10, 'max_leaves': 282, 'grow_policy': 'depthwise', 'learning_rate': 1.7879717055987567, 'gamma': 1.4354330860078335, 'subsample': 0.17676433268934488, 'colsample_bytree': 0.24637066572904587, 'colsample_bylevel': 0.6342601981014013, 'colsample_bynode': 0.8516784459352493}

Training model
{'n_estimators': 

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 64, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 0.0, 'subsample': 1.0, 'colsample_bytree': 0.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.0}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 64, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 0.0, 'subsample': 1.0, 'colsample_bytree': 0.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.0}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 64, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 0.0, 'subsample': 1.0, 'colsample_bytree': 0.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.0}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 64, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 0.0, 'subsample': 1.0, 'colsample_bytree': 0.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.0}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 64, 'gro

Training model
{'n_estimators': 9, 'max_depth': 9, 'max_leaves': 1304, 'grow_policy': 'lossguide', 'learning_rate': 1.035516062732786, 'gamma': 11.651569865643978, 'subsample': 0.39984926767647266, 'colsample_bytree': 0.5606478620320559, 'colsample_bylevel': 0.10095539689064026, 'colsample_bynode': 0.07605439983308315}

Training model
{'n_estimators': 8, 'max_depth': 8, 'max_leaves': 196, 'grow_policy': 'depthwise', 'learning_rate': 0.1954847559798509, 'gamma': 13.138851802796125, 'subsample': 0.6726743504405022, 'colsample_bytree': 0.9430814571678638, 'colsample_bylevel': 0.5273707546293736, 'colsample_bynode': 0.8318993551656604}

Training model
{'n_estimators': 44, 'max_depth': 9, 'max_leaves': 69, 'grow_policy': 'lossguide', 'learning_rate': 1.3113806979730724, 'gamma': 19.25405204296112, 'subsample': 0.023318310268223286, 'colsample_bytree': 0.263179799541831, 'colsample_bylevel': 0.6917098853737116, 'colsample_bynode': 0.8961779372766614}

Training model
{'n_estimators': 15, 'max

...


Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 64, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 20.0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 1.0}

Training model
{'n_estimators': 6, 'max_depth': 1, 'max_leaves': 1161, 'grow_policy': 'lossguide', 'learning_rate': 0.16177220783504806, 'gamma': 18.94996977497796, 'subsample': 0.04361393597246316, 'colsample_bytree': 0.7365706017291259, 'colsample_bylevel': 0.8392610855299387, 'colsample_bynode': 0.32342181113266494}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 2048, 'grow_policy': 'depthwise', 'learning_rate': 2.0, 'gamma': 20.0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 1.0}

Training model
{'n_estimators': 88, 'max_depth': 9, 'max_leaves': 794, 'grow_policy': 'depthwise', 'learning_rate': 1.0889793215975485, 'gamma': 18.744099932764307, 'subsample': 0.13404839547645891, 'colsample_b


Training model
{'n_estimators': 18, 'max_depth': 3, 'max_leaves': 1047, 'grow_policy': 'lossguide', 'learning_rate': 0.5435651738848537, 'gamma': 18.60521651804447, 'subsample': 0.686706087552011, 'colsample_bytree': 0.4351745117455721, 'colsample_bylevel': 0.058822798542678356, 'colsample_bynode': 0.8474182551726699}

Training model
{'n_estimators': 70, 'max_depth': 2, 'max_leaves': 1206, 'grow_policy': 'depthwise', 'learning_rate': 1.6437397051975131, 'gamma': 19.357905108481646, 'subsample': 0.18241268675774336, 'colsample_bytree': 0.014711706899106503, 'colsample_bylevel': 0.2790142586454749, 'colsample_bynode': 0.11880306247621775}

Training model
{'n_estimators': 39, 'max_depth': 9, 'max_leaves': 195, 'grow_policy': 'lossguide', 'learning_rate': 0.021769232247024775, 'gamma': 10.143702868372202, 'subsample': 0.16837050020694733, 'colsample_bytree': 0.4941681595519185, 'colsample_bylevel': 0.13682643603533506, 'colsample_bynode': 0.8187597673386335}

Training model
{'n_estimators


Training model
{'n_estimators': 12, 'max_depth': 9, 'max_leaves': 1820, 'grow_policy': 'depthwise', 'learning_rate': 1.346493626208976, 'gamma': 11.128360517323017, 'subsample': 0.8747398713603616, 'colsample_bytree': 0.7398711498826742, 'colsample_bylevel': 0.0916577447205782, 'colsample_bynode': 0.12758140731602907}

Training model
{'n_estimators': 65, 'max_depth': 1, 'max_leaves': 424, 'grow_policy': 'depthwise', 'learning_rate': 1.292012154320255, 'gamma': 16.293261870741844, 'subsample': 0.34361441992223263, 'colsample_bytree': 0.07742267288267612, 'colsample_bylevel': 0.08886190596967936, 'colsample_bynode': 0.06164179928600788}

Training model
{'n_estimators': 81, 'max_depth': 9, 'max_leaves': 1423, 'grow_policy': 'lossguide', 'learning_rate': 1.9934038971643895, 'gamma': 4.2384253814816475, 'subsample': 0.07358475588262081, 'colsample_bytree': 0.9320258107036352, 'colsample_bylevel': 0.8048136914148927, 'colsample_bynode': 0.5434928061440587}

Training model
{'n_estimators': 1


Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 2048, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 20.0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.3046274590230857}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 2048, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 20.0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.30461534301811793}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 2048, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 20.0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.30462414291926027}

Training model
{'n_estimators': 100, 'max_depth': 10, 'max_leaves': 2048, 'grow_policy': 'lossguide', 'learning_rate': 2.0, 'gamma': 20.0, 'subsample': 1.0, 'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'colsample_bynode': 0.3046183904873803}

T


Training model
{'n_estimators': 86, 'max_depth': 5, 'max_leaves': 1239, 'grow_policy': 'depthwise', 'learning_rate': 0.061102688405662776, 'gamma': 18.634355273097754, 'subsample': 0.4167916066944599, 'colsample_bytree': 0.9995599035173655, 'colsample_bylevel': 0.018251542001962662, 'colsample_bynode': 0.5652335761114955}

Training model
{'n_estimators': 65, 'max_depth': 1, 'max_leaves': 824, 'grow_policy': 'lossguide', 'learning_rate': 0.21843036924488843, 'gamma': 6.674983724951744, 'subsample': 0.6639381488785148, 'colsample_bytree': 0.27897598687559366, 'colsample_bylevel': 0.8071034904569387, 'colsample_bynode': 0.805176978930831}

Training model
{'n_estimators': 99, 'max_depth': 9, 'max_leaves': 536, 'grow_policy': 'depthwise', 'learning_rate': 1.9186313615646213, 'gamma': 8.448912966996431, 'subsample': 0.8295325748622417, 'colsample_bytree': 0.6587322652339935, 'colsample_bylevel': 0.6072524609044194, 'colsample_bynode': 0.6419218825176358}

Training model
{'n_estimators': 3, 

In [ ]:
pd.DataFrame(performance).pivot(index=['Model', 'Q', 'Acquisition Function'], 
                                columns='Dataset', 
                                values='Performance')

In [3]:
pd.DataFrame(performance).pivot(index=['Model', 'Q', 'Acquisition Function'], 
                                columns='Dataset', 
                                values='Performance')

Dataset                                     adult       car  diabetes  \
Model     Q  Acquisition Function                                       
matern0_5 4  Expected Improvement        0.814617  0.960830  0.497351   
             Probability of Improvement  0.823196  0.915566  0.481748   
             Upper Confidence Bound      0.814147  0.982453  0.504090   
          16 Expected Improvement        0.822187  0.826141  0.500142   
             Probability of Improvement  0.819326  0.843447  0.487830   
             Upper Confidence Bound      0.814147  0.982453  0.504090   
matern1_5 4  Expected Improvement        0.823168  0.852369  0.492654   
             Probability of Improvement  0.818488  0.960830  0.494494   
             Upper Confidence Bound      0.817371  0.982453  0.504090   
          16 Expected Improvement        0.817580  0.952136  0.497630   
             Probability of Improvement  0.818503  0.872565  0.494325   
             Upper Confidence Bound      0.814147  0.922078  0.486961   
matern2_5 4  Expected Improvement        0.820814  0.842728  0.488676   
             Probability of Improvement  0.834191  0.914665  0.490354   
             Upper Confidence Bound      0.817371  0.740059  0.500032   
          16 Expected Improvement        0.817821  0.907726  0.494325   
             Probability of Improvement  0.823856  0.836722  0.497106   
             Upper Confidence Bound      0.814147  0.842728  0.510523   

Dataset                                  drybeans      iris  
Model     Q  Acquisition Function                            
matern0_5 4  Expected Improvement        0.904329  0.951606  
             Probability of Improvement  0.893526  0.936260  
             Upper Confidence Bound      0.880927  0.944643  
          16 Expected Improvement        0.907098  0.944925  
             Probability of Improvement  0.901387  0.947166  
             Upper Confidence Bound      0.903310  0.944573  
matern1_5 4  Expected Improvement        0.893975  0.945786  
             Probability of Improvement  0.900484  0.944017  
             Upper Confidence Bound      0.903310  0.942912  
          16 Expected Improvement        0.891727  0.921148  
             Probability of Improvement  0.897827  0.901619  
             Upper Confidence Bound      0.880927  0.935785  
matern2_5 4  Expected Improvement        0.893009  0.942912  
             Probability of Improvement  0.892398  0.944599  
             Upper Confidence Bound      0.903310  0.942912  
          16 Expected Improvement        0.902150  0.942912  
             Probability of Improvement  0.891821  0.944343  
             Upper Confidence Bound      0.893441  0.942912

In [4]:
pd.DataFrame(performance).to_csv('results.csv', index=False)